In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [2]:
data = pd.read_csv('../input/first-gop-debate-twitter-sentiment/Sentiment.csv')
print(data.head())

   id               candidate  candidate_confidence relevant_yn  \
0   1  No candidate mentioned                   1.0         yes   
1   2            Scott Walker                   1.0         yes   
2   3  No candidate mentioned                   1.0         yes   
3   4  No candidate mentioned                   1.0         yes   
4   5            Donald Trump                   1.0         yes   

   relevant_yn_confidence sentiment  sentiment_confidence     subject_matter  \
0                     1.0   Neutral                0.6578  None of the above   
1                     1.0  Positive                0.6333  None of the above   
2                     1.0   Neutral                0.6629  None of the above   
3                     1.0  Positive                1.0000  None of the above   
4                     1.0  Positive                0.7045  None of the above   

   subject_matter_confidence candidate_gold  ... relevant_yn_gold  \
0                     1.0000            NaN  ..

In [3]:
data = data[['text','sentiment']]

In [4]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2500
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

4472
16986


In [5]:
model = Sequential()
model.add(Embedding(max_fatures, 256, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

2022-11-25 04:36:26.282479: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-25 04:36:26.432676: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-25 04:36:26.433566: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-25 04:36:26.435417: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 29, 256)           640000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 29, 256)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 2)                 514       
Total params: 1,165,826
Trainable params: 1,165,826
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7510, 29) (7510, 2)
(3219, 29) (3219, 2)


In [7]:
history = model.fit(X_train, Y_train, epochs = 10, batch_size=32, verbose = 2)

Epoch 1/10


2022-11-25 04:36:29.302706: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


235/235 - 35s - loss: 0.4294 - accuracy: 0.8234
Epoch 2/10
235/235 - 32s - loss: 0.3085 - accuracy: 0.8724
Epoch 3/10
235/235 - 35s - loss: 0.2601 - accuracy: 0.8952
Epoch 4/10
235/235 - 36s - loss: 0.2202 - accuracy: 0.9144
Epoch 5/10
235/235 - 37s - loss: 0.1911 - accuracy: 0.9234
Epoch 6/10
235/235 - 45s - loss: 0.1635 - accuracy: 0.9333
Epoch 7/10
235/235 - 36s - loss: 0.1416 - accuracy: 0.9423
Epoch 8/10
235/235 - 35s - loss: 0.1247 - accuracy: 0.9489
Epoch 9/10
235/235 - 34s - loss: 0.1150 - accuracy: 0.9542
Epoch 10/10
235/235 - 35s - loss: 0.0998 - accuracy: 0.9571


In [34]:
df = data['text']
print(df[:1])

1      scottwalker didnt catch the full gopdebate l...
Name: text, dtype: object


In [8]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
print(twt)
twt = tokenizer.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
sentiment = model.predict(twt,batch_size=1,verbose = 0)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

['Meetings: Because none of us is as dumb as all of us.']
negative


In [39]:
twt2 = ['he is in the right state']
print(twt2)
twt2 = tokenizer.texts_to_sequences(twt2)
twt2 = pad_sequences(twt2, maxlen=28, dtype='int32', value=0)
sentiment = model.predict(twt2,batch_size=1,verbose = 0)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

['he is in the right state']
positive
